In [1]:
import os
import pandas as pd

from pathlib import Path
from datetime import datetime

from src.models.LogisticRegression import LogisticRegressionClassifier
from src.models.SVM import SVMClassifier
from src.models.RandomForest import RFClassifier
from src.models.MLP import MLP
from src.models.KN import KN
from src.models.DecisionTree import DT
from sklearn.model_selection import StratifiedKFold

In [2]:
# Variables used throughout the notebook
project_root_dir = Path(os.path.abspath('')).resolve().parents[0]
current_notebook = "Training"

# output path for images
image_folder_path = os.path.join(project_root_dir, "notebooks", current_notebook)
os.makedirs(image_folder_path, exist_ok=True)

In [3]:
def display_score(name, score):
    score_string = name + ': '
    for key in score:
        score_string += key + '={:.4f}'.format(score[key]) + '\t'

    print(score_string)

In [4]:
def build_dict_from_results(name, score):
    score['name'] = name
    return score

## Data

In [5]:
# import the data
data_path = os.path.join(project_root_dir, 'data', 'processed')

data = pd.read_csv(os.path.join(data_path, 'training_data.csv'))
labels = pd.read_csv(os.path.join(data_path, 'training_labels.csv'))

In [6]:
data.describe()

,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,margin10,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
count,792.000000,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,...,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02,7.920000e+02
mean,0.000000,3.364312e-17,1.300867e-16,1.014901e-16,-5.719331e-17,1.166295e-16,-5.607187e-17,-1.794300e-17,-5.831474e-17,6.055762e-17,...,3.252168e-17,6.728624e-17,-2.018587e-17,5.382900e-17,-2.063445e-16,1.087794e-16,-1.570012e-17,2.018587e-17,4.934325e-17,-1.401797e-16
std,1.000632,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,...,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00,1.000632e+00
min,-0.881617,-7.342717e-01,-1.240112e+00,-8.066802e-01,-7.626971e-01,-7.345422e-01,-1.095629e+00,-3.792727e-01,-8.283998e-01,-1.154050e+00,...,-5.656381e-01,-2.606863e-01,-7.002438e-01,-4.547012e-01,-1.046938e+00,-2.332711e-01,-2.344170e-01,-5.220187e-01,-6.579067e-01,-8.594620e-01
25%,-0.781026,-6.841764e-01,-7.047405e-01,-6.046400e-01,-6.580867e-01,-7.345422e-01,-7.569981e-01,-3.792727e-01,-6.039509e-01,-7.929953e-01,...,-5.656381e-01,-2.606863e-01,-6.572363e-01,-4.547012e-01,-7.272258e-01,-2.332711e-01,-2.344170e-01,-5.220187e-01,-6.579067e-01,-8.166401e-01
50%,-0.378613,-4.336742e-01,-3.223209e-01,-3.352186e-01,-3.442556e-01,-4.358417e-01,-3.054330e-01,-3.792727e-01,-1.550532e-01,-1.911757e-01,...,-4.886857e-01,-2.606863e-01,-4.423305e-01,-4.161063e-01,-2.157383e-01,-2.332711e-01,-2.344170e-01,-4.211254e-01,-4.445670e-01,-3.030405e-01
75%,0.426161,2.802158e-01,4.424790e-01,2.035897e-01,1.788499e-01,3.202531e-01,4.847627e-01,-3.792727e-01,6.939565e-02,5.309956e-01,...,6.919588e-02,-2.606863e-01,2.562342e-01,-1.075048e-01,3.597179e-01,-2.332711e-01,-2.344170e-01,8.339310e-02,2.664437e-01,4.246247e-01
max,3.444026,4.526119e+00,3.807779e+00,5.052796e+00,5.200577e+00,5.202191e+00,4.209929e+00,1.077247e+01,6.803321e+00,4.863900e+00,...,6.205956e+00,9.794375e+00,6.908617e+00,7.453904e+00,5.922832e+00,9.088584e+00,1.268766e+01,9.189677e+00,5.670489e+00,4.276973e+00


In [7]:
labels.describe()

,species
count,792.000000
mean,49.000000
std,28.595439
min,0.000000
25%,24.000000
50%,49.000000
75%,74.000000
max,98.000000


In [8]:
s_k_fold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

## Training

In [9]:
score_list = []

### Logistic regression

In [10]:
from src.models.LogisticRegression import LogisticRegressionClassifier

lr_clf = LogisticRegressionClassifier(s_k_fold)

Logistic Regression:Creating


In [11]:
# print results without any optimization
lr_clf.cross_validate(data, labels, optimized=False)
display_score(lr_clf.name, lr_clf.get_score())

score_list.append(build_dict_from_results(lr_clf.name, lr_clf.get_score()))

Logistic Regression:Initialization
Logistic Regression: f1=0.9843	precision=0.9848	recall=0.9874	accuracy=0.9874	log_loss=0.1454	


In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
lr_clf.optimize(data, labels)

In [12]:
lr_clf.cross_validate(data, labels, optimized=True)
display_score(lr_clf.name, lr_clf.get_score())

score_list.append(build_dict_from_results(lr_clf.name, lr_clf.get_score()))

Logistic Regression:Initialization
Logistic Regression:C:390.01768308021974 penalty:l2 solver:lbfgs 
Logistic Regression: f1=0.9843	precision=0.9848	recall=0.9874	accuracy=0.9874	log_loss=0.0542	


### SVM

In [13]:
from src.models.SVM import SVMClassifier

svm_clf = SVMClassifier(s_k_fold)

SVM:Creating


In [14]:
# print results without any optimization
svm_clf.cross_validate(data, labels, optimized=False)
display_score(svm_clf.name, svm_clf.get_score())

score_list.append(build_dict_from_results(svm_clf.name, svm_clf.get_score()))

SVM:Initialization
SVM: f1=0.9749	precision=0.9783	recall=0.9785	accuracy=0.9785	log_loss=2.5780	


In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
svm_clf.optimize(data, labels)

In [15]:
svm_clf.cross_validate(data, labels, optimized=True)
display_score(svm_clf.name, svm_clf.get_score())

score_list.append(build_dict_from_results(svm_clf.name, svm_clf.get_score()))

SVM:Initialization
SVM:probability:True kernel:poly gamma:auto degree:1 C:1.7782794100389228e+74 
SVM: f1=0.9808	precision=0.9834	recall=0.9836	accuracy=0.9836	log_loss=2.5311	


### Random forest

In [16]:
from src.models.RandomForest import RFClassifier

rf_clf = RFClassifier(s_k_fold)

Random Forest:Creating


In [17]:
# print results without any optimization
rf_clf.cross_validate(data, labels, optimized=False)
display_score(rf_clf.name, rf_clf.get_score())

score_list.append(build_dict_from_results(rf_clf.name, rf_clf.get_score()))

Random Forest:Initialization
Random Forest: f1=0.9709	precision=0.9785	recall=0.9735	accuracy=0.9735	log_loss=0.9300	


In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
rf_clf.optimize(data, labels)

In [ ]:
rf_clf.cross_validate(data, labels, optimized=True)
display_score(rf_clf.name, rf_clf.get_score())

score_list.append(build_dict_from_results(rf_clf.name, rf_clf.get_score()))

Random Forest:Initialization
Random Forest:random_state:42 n_estimators:4000 min_samples_split:2 min_samples_leaf:1 max_features:None max_depth:12 criterion:entropy 


### Multi layer perceptron

In [ ]:
from src.models.MLP import MLP

mlp_clf = MLP(s_k_fold)

In [ ]:
# print results without any optimization
mlp_clf.cross_validate(data, labels, optimized=False)
display_score(mlp_clf.name, mlp_clf.get_score())

score_list.append(build_dict_from_results(mlp_clf.name, mlp_clf.get_score()))

In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
mlp_clf.optimize(data, labels)

In [ ]:
mlp_clf.cross_validate(data, labels, optimized=True)
display_score(mlp_clf.name, mlp_clf.get_score())

score_list.append(build_dict_from_results(mlp_clf.name, mlp_clf.get_score()))

### KNeighbors 

In [ ]:
from src.models.KN import KN

kn_clf = KN(s_k_fold)

In [ ]:
# print results without any optimization
kn_clf.cross_validate(data, labels, optimized=False)
display_score(kn_clf.name, kn_clf.get_score())

score_list.append(build_dict_from_results(kn_clf.name, kn_clf.get_score()))

In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
kn_clf.optimize(data, labels)

In [ ]:
kn_clf.cross_validate(data, labels, optimized=True)
display_score(kn_clf.name, mlp_clf.get_score())

score_list.append(build_dict_from_results(kn_clf.name, kn_clf.get_score()))

### Decision Tree

In [ ]:
from src.models.DecisionTree import DT

dt_clf = DT(s_k_fold)

In [ ]:
# print results without any optimization
dt_clf.cross_validate(data, labels, optimized=False)
display_score(dt_clf.name, dt_clf.get_score())

score_list.append(build_dict_from_results(dt_clf.name, dt_clf.get_score()))

In [ ]:
# This method will search the best set of hyperparameters with a RandomSearch. 
# The hyper-parameters have been fitted to have to the best range
dt_clf.optimize(data, labels)

In [ ]:
dt_clf.cross_validate(data, labels, optimized=True)
display_score(kn_clf.name, mlp_clf.get_score())

score_list.append(build_dict_from_results(dt_clf.name, dt_clf.get_score()))